In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup

import re
import time

In [2]:
def typekeySlowly(dst, value):
    for c in value:
        dst.send_keys(c)
        time.sleep(0.3)
    time.sleep(1)
def init(googledrive_file):
    options = webdriver.ChromeOptions()
#     options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("--disable-gpu")
    
    driver = webdriver.Chrome(googledrive_file, options=options)
    
    driver.get("https://eclass3.cau.ac.kr/")
    driver.find_element_by_xpath('//*[@id="gnb"]/ul/li[3]/a').click()
    
    input("로그인이 끝나셨으면 엔터를 쳐주세요!")

    return driver

In [13]:
def metaData(driver):
    classNames = []
    classNumbers = []

    # dashboard에서 각 과목별 이름과 uri 추출
#     driver.get("https://eclass3.cau.ac.kr/")
#     driver.find_element_by_xpath('//*[@id="gnb"]/ul/li[3]/a').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.select('#DashboardCard_Container > div > div')
    for card in cards:
        # 이름 추출
        try:
            label = card.attrs['aria-label']
            label = re.compile(r'^(.*) \d\d.+$').search(label).group(1)
        except:
            label = card.attrs['aria-label']
        classNames += [label]

        # uri 추출
        cardId = card.attrs['data-reactid']
        cardId = re.compile(r'\$(\d+)$').search(cardId).group(1)
        cardId = int(cardId)
        classNumbers += [cardId]
    return classNames, classNumbers

In [14]:
def classes(driver):
    # classes 계층: 수업(소프트웨어공학) > 주차(1주차) > 수업(1.1 SE Introduction)
    # classes > sections > lectures
    classArr =[]
    for num in classNumbers:
        driver.get("https://eclass3.cau.ac.kr/courses/"+str(num)+'/external_tools/2')
        time.sleep(2)
        driver.switch_to.frame(driver.find_element_by_id('tool_content'))
        time.sleep(2)
        
        sectionArr = []
        for section in driver.find_elements_by_xpath("//*[@class='xn-section']"):    
            lectureArr = []    
            for lecture in section.find_elements_by_class_name('xn-component-item'):
                sub = lecture.find_element_by_class_name('xnci-description-component-type-icon-container')
                if "movie" in sub.get_attribute('class'): lectureArr += [True]
                else: lectureArr += [False]
            sectionArr.append(lectureArr)     
        classArr.append(sectionArr)
    return classArr

In [15]:
# topest somthing > iframe#tool_content(출발지) > iframe.xnvc-video-frame(동영상들) > iframe(목적지)

from selenium.common.exceptions import NoSuchElementException

def extract(driver, lectures, className, numSection):
    time.sleep(2)
    videos = driver.find_elements_by_class_name('xnu-component-container')
    isSpecial = len(videos) != 1
    
    videos_id = []
    for idx, video in enumerate(videos): # 동영상들 순회
        if lectures[idx] == False: continue
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'xnct-title')))
        video_name = video.find_element_by_class_name('xnct-title').text
        
        try:
            driver.switch_to.frame(video.find_element_by_tag_name('iframe'))
            src = driver.find_element_by_tag_name('iframe').get_attribute('src')
            video_id = re.compile(r'https://.*/(.*)\?').search(src).group(1)
            videos_id += [(video_id, video_name)]
            
            driver.switch_to.parent_frame()
        except NoSuchElementException:
            print(className+" "+str(numSection+1)+"주차에 영상을 추출할 수가 없습니다.")
        
    return isSpecial, videos_id

In [16]:
def unfold(section):
    folder = section.find_element_by_class_name('xnslh-section-title')
    driver.execute_script("arguments[0].click();", folder) 
    
def search(driver, classArr, classNumbers, classNames):
    time.sleep(5)
    video_class = []
    for numClass, sections in enumerate(classArr):
        driver.get("https://eclass3.cau.ac.kr/courses/"+str(classNumbers[numClass])+'/external_tools/2')

        video_sections = []
        for numSection, lectures in enumerate(sections):
            isSpecial = False
            video_lectures = []

            for numLecture, islecture in enumerate(lectures):
                time.sleep(3)
                if isSpecial: break
                if not islecture: continue
                try:
                    driver.switch_to.frame(driver.find_element_by_id('tool_content'))
                except:
                    driver.refresh()
                    time.sleep(5)
                    driver.switch_to.frame(driver.find_element_by_id('tool_content'))
                finally:

                    section = driver.find_elements_by_class_name('xn-section')[numSection]
                    lecture = section.find_elements_by_class_name('xn-component-item')[numLecture]
                    unfold(section)
                    driver.execute_script("arguments[0].click();", lecture) 
                    isSpecial, videos_id = extract(driver, lectures, classNames[numClass], numSection)

                    if not isSpecial: video_lectures.extend(videos_id)
                    else: video_lectures = videos_id
                    driver.back()
            video_sections.append(video_lectures)
        video_class.append(video_sections)
    return video_class

In [ ]:
import requests
import os
import threading, requests, time

def createFolder(directory):
    try:
        if not os.path.exists(directory): os.makedirs(directory)
    except OSError:
        print ('Create '+ directory +' Folder Error')

def download(className, root_dir, numSection, video_id, video_name):
    createFolder(root_dir + "/" + className)
    createFolder(root_dir + "/" + className + "/" + str(numSection+1)+"주차")
    
    if len(video_name) > 10: video_name = video_name[:10]
    
    for idx in range(10):
        try:
            url = "https://cau.commonscdn.com/contents"+str(idx)+"/cau1000001/"+video_id+"/contents/media_files/mobile/ssmovie.mp4"
            myfile = requests.get(url)
            code = open(root_dir+"/"+className+"/"+str(numSection+1)+"주차"+"/"+video_name+".mp4", 'wb').write(myfile.content)
            
            if code != 1238: break        
        except (OSError, IOError) as e:
            print(e)
            break
        except:
            print(code, url)
    if idx == 99:
        print("Download Error at " + className +", week"+ str(numSection+1) +", code: "+ video_id)
        
def download_all(vidoes_class, root_dir, classNames):
    sema = threading.Semaphore(10)
    
    for sections, className in zip(videos_class, classNames):
        for numSection, section in enumerate(sections):
            for lecture in section:
                sema.acquire()
                time.sleep(1)
                download(className, root_dir, numSection, lecture[0], lecture[1])
                sema.release()

In [8]:
from tkinter import *
from tkinter import filedialog
import sys

root = Tk()
root_dir = filedialog.askdirectory(title = "저장할 위치를 선택해주세요")
googledrive_file = filedialog.askopenfilename(title = "chromedriver를 선택해주세요", filetypes =[("chrome driver", "*.exe")])
if(root_dir == "" or googledrive_file == ""):
    root.destroy()
    sys.exit("폴더나 파일이 지정되지 않았습니다!!!!!")
root.destroy()

In [69]:
print("추출을 진행 중입니다")
driver = init(googledrive_file)
print("가상 크롬을 생성했습니다. 절대 건드리지 마세요!")
classNames, classNumbers = metaData(driver)
print("메타 데이터를 추출했습니다.")
classArr = classes(driver)
print("강의 정보를 추출했습니다.")
videos_class = search(driver, classArr, classNumbers, classNames)
print("영상별 ID를 추출했습니다.")
print("영상을 다운받는 중입니다. 오래 걸립니다:)")
download_all(videos_class, root_dir, classNames)
print("모두 끝났습니다!!")